In [7]:
# test reading in 10x data with AnnData
import pandas as pd
import numpy as np
import scanpy as sc
from glob import glob
import os

from matplotlib import pyplot as plt
import seaborn as sns
sns.set_palette(sns.color_palette("Spectral"))
sns.set_style("whitegrid")

from matplotlib.ticker import StrMethodFormatter
# ax.xaxis.set_major_formatter(StrMethodFormatter("{x:.2f}"))

In [22]:
os.chdir('c:\\Users\\randymi\\Desktop\\Suerat Work')

In [79]:
adata_bc = pd.read_csv("data_extract/GSE154775/GSM4679492_HS1/features.tsv", header=None, delimiter = "\t")

In [101]:
adata_bc[1]

0          MIR1302-10
1             FAM138A
2               OR4F5
3        RP11-34P13.7
4        RP11-34P13.8
             ...     
32733      AC145205.1
32734           BAGE5
32735      CU459201.1
32736      AC002321.2
32737      AC002321.1
Name: 1, Length: 32738, dtype: object

In [112]:
# Function is beyond broken
# scanpy.read_10x_mtx("data_extract/GSE154775/GSM4679492_HS1")

def read_dir(data_path):
    matrix_path = os.path.join(data_path, 'matrix.mtx')
    barcodes_path = os.path.join(data_path, 'barcodes.tsv')
    features_path = os.path.join(data_path, 'features.tsv')
    
    adata = sc.read_mtx(matrix_path)
    adata_bc = pd.read_csv(barcodes_path, header=None, delimiter = "\t")
    adata_features = pd.read_csv(features_path, header=None, delimiter = '\t')
    print(adata_features)
    
    adata = adata.T
    adata.obs['cell_id'] = adata_bc
    adata.var['gene_name'] = adata_features[1].values
    adata.var.index = adata.var['gene_name']
    
    return adata

In [114]:
adata = sc.read_mtx("data_extract/GSE154775/GSM4679492_HS1/matrix.mtx")

In [118]:
adata_bc = pd.read_csv("data_extract/GSE154775/GSM4679492_HS1/features.tsv", header = None, delimiter = '\t')
adata_bc

,0,1
0,ENSG00000243485,MIR1302-10
1,ENSG00000237613,FAM138A
2,ENSG00000186092,OR4F5
3,ENSG00000238009,RP11-34P13.7
4,ENSG00000239945,RP11-34P13.8
...,...,...
32733,ENSG00000215635,AC145205.1
32734,ENSG00000268590,BAGE5
32735,ENSG00000251180,CU459201.1
32736,ENSG00000215616,AC002321.2


In [116]:
adata.var

""
0
1
2
3
4
...
5631
5632
5633
5634


In [113]:
dff = read_dir("data_extract/GSE154775/GSM4679492_HS1")

                     0             1
0      ENSG00000243485    MIR1302-10
1      ENSG00000237613       FAM138A
2      ENSG00000186092         OR4F5
3      ENSG00000238009  RP11-34P13.7
4      ENSG00000239945  RP11-34P13.8
...                ...           ...
32733  ENSG00000215635    AC145205.1
32734  ENSG00000268590         BAGE5
32735  ENSG00000251180    CU459201.1
32736  ENSG00000215616    AC002321.2
32737  ENSG00000215611    AC002321.1

[32738 rows x 2 columns]


In [103]:
df.var_names_make_unique()
df.obs_names_make_unique()

In [104]:
df.var["mt"] = df.var_names.str.startswith("MT-")
df.var["ribo"] = df.var_names.str.startswith(("RPS", "RPL"))

In [111]:
type(df.var)

pandas.core.frame.DataFrame

In [44]:
pd.read_csv("data_extract/GSE154775/GSM4679492_HS1/features.tsv", delimiter = '\t', header=None)

,0,1
0,ENSG00000243485,MIR1302-10
1,ENSG00000237613,FAM138A
2,ENSG00000186092,OR4F5
3,ENSG00000238009,RP11-34P13.7
4,ENSG00000239945,RP11-34P13.8
...,...,...
32733,ENSG00000215635,AC145205.1
32734,ENSG00000268590,BAGE5
32735,ENSG00000251180,CU459201.1
32736,ENSG00000215616,AC002321.2


# Preprocessing Steps
1. load data
2. QC, for each sample, filter for cells with:
    a. 300 < Unique Genes < 6000
    b. total RNA count < 40000
    c. Michtocondrial DNA < 20%
    d. Perform doublet detection and drop doublet cells
    e. merge by concatenate
    f. normalize

